# Modelos Customizados

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# loading fashion mnist
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [2]:
# data normalization
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000].copy(), y_train_full[5000:].copy()

In [3]:
mnist_classes = ['T-shirt/top', 'Trouser/pants', 'Pullover shirt', 'Dress', 
                 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

In [4]:
y_valid_d = tf.constant(pd.get_dummies(y_valid).values, tf.float32)
y_train_d = tf.constant(pd.get_dummies(y_train).values, tf.float32)

In [5]:
y_train_d

<tf.Tensor: id=1, shape=(55000, 10), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [6]:
X_input = keras.layers.Input(shape=[28, 28])

h0    = keras.layers.Flatten()(X_input)
h1    = keras.layers.Dense(300, activation="relu")(h0)
h2    = keras.layers.Dense(200, activation="relu")(h1)
h3    = keras.layers.Dense(100, activation="relu")(h2)
h4    = keras.layers.Dense( 50, activation="relu")(h3)
yhat  = keras.layers.Dense( 10, activation="sigmoid")(h4)
model = keras.models.Model(inputs=[X_input], outputs=[yhat])

### Funções de perda

In [7]:
# qd usa batch norm, essa loss n funciona bem, pelo menos, nas 1as poucas epocas
def cosine_loss(v1, v2):
    # y_true, y_pred
    norm_v1 = tf.nn.l2_normalize(v1, 1)        
    norm_v2 = tf.nn.l2_normalize(v2, 1)
    return 1 - tf.matmul(norm_v1, norm_v2, transpose_b=True)

In [18]:
sgd = keras.optimizers.SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss = cosine_loss, optimizer = sgd, metrics = ["accuracy"])

In [9]:
h = model.fit(X_train, y_train_d, epochs = 10, shuffle = True,
             validation_data = (X_valid, y_valid_d))

W0923 12:53:29.738394 4619273664 training_utils.py:1211] When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.


Epoch 1/10


W0923 12:53:30.013667 4619273664 deprecation.py:323] From /Users/marcocristo/.local/lib/python3.6/site-packages/tensorflow_core/python/ops/math_grad.py:1393: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0923 12:53:30.059385 4619273664 deprecation.py:323] From /Users/marcocristo/.local/lib/python3.6/site-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:468: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.


55000/55000 [==============================] - 7s 125us/sample - loss: 0.6838 - accuracy: 0.3071 - val_loss: 0.6838 - val_accuracy: 0.3940
Epoch 2/10
55000/55000 [==============================] - 6s 110us/sample - loss: 0.6835 - accuracy: 0.5658 - val_loss: 0.6836 - val_accuracy: 0.6090
Epoch 3/10
55000/55000 [==============================] - 6s 111us/sample - loss: 0.6833 - accuracy: 0.6143 - val_loss: 0.6835 - val_accuracy: 0.6140
Epoch 4/10
55000/55000 [==============================] - 7s 121us/sample - loss: 0.6833 - accuracy: 0.6609 - val_loss: 0.6832 - val_accuracy: 0.6812
Epoch 5/10
55000/55000 [==============================] - 7s 118us/sample - loss: 0.6832 - accuracy: 0.6853 - val_loss: 0.6831 - val_accuracy: 0.7098
Epoch 6/10
55000/55000 [==============================] - 7s 119us/sample - loss: 0.6832 - accuracy: 0.7114 - val_loss: 0.6832 - val_accuracy: 0.7618
Epoch 7/10
55000/55000 [==============================] - 6s 115us/sample - loss: 0.6832 - accuracy: 0.7404 - v

#### Salvando função de perda customizada

In [10]:
model.save("model_cosine_loss.h5")

In [11]:
# Na hora de ler, voce precisa forncecer um dicionario custom_objects com a funcao de perda
model = keras.models.load_model("model_cosine_loss.h5", 
                                custom_objects = {"cosine_loss": cosine_loss})

### Funções de perda com parâmetro

In [12]:
def gere_cosine_loss(classw = tf.constant(1, shape = (10,), dtype = tf.float32)):
    def cosine_loss(v1, v2):
        # y_true, y_pred
        v2 = classw * v2
        norm_v1 = tf.nn.l2_normalize(v1, 1)        
        norm_v2 = tf.nn.l2_normalize(v2, 1)
        return 1 - tf.matmul(norm_v1, norm_v2, transpose_b=True)
    return cosine_loss

In [13]:
model.compile(loss = gere_cosine_loss(), optimizer = "adam", metrics = ["accuracy"])

In [14]:
h = model.fit(X_train, y_train_d, epochs = 10, 
             validation_data = (X_valid, y_valid_d))

W0923 12:54:35.504133 4619273664 training_utils.py:1211] When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.


Epoch 1/10
55000/55000 [==============================] - 8s 140us/sample - loss: 0.6834 - accuracy: 0.5756 - val_loss: 0.6833 - val_accuracy: 0.7272
Epoch 2/10
55000/55000 [==============================] - 7s 136us/sample - loss: 0.6831 - accuracy: 0.7064 - val_loss: 0.6832 - val_accuracy: 0.7338
Epoch 3/10
55000/55000 [==============================] - 7s 126us/sample - loss: 0.6831 - accuracy: 0.7297 - val_loss: 0.6830 - val_accuracy: 0.7742
Epoch 4/10
55000/55000 [==============================] - 7s 130us/sample - loss: 0.6830 - accuracy: 0.7409 - val_loss: 0.6830 - val_accuracy: 0.7780
Epoch 5/10
55000/55000 [==============================] - 7s 130us/sample - loss: 0.6830 - accuracy: 0.7605 - val_loss: 0.6830 - val_accuracy: 0.7948
Epoch 6/10
55000/55000 [==============================] - 7s 126us/sample - loss: 0.6830 - accuracy: 0.7695 - val_loss: 0.6829 - val_accuracy: 0.7860
Epoch 7/10
55000/55000 [==============================] - 7s 129us/sample - loss: 0.6830 - accuracy:

In [15]:
model.save("model_cosine_loss.h5")

In [16]:
# Na hora de ler, voce precisa forncecer um dicionario custom_objects com a funcao de perda
model = keras.models.load_model("model_cosine_loss.h5", 
                                custom_objects = {"cosine_loss": gere_cosine_loss()})

Note que a função salva foi a usada no modelo mas não a configurável! Logo, o parâmetro não foi salvo. Para contornar isso, é melhor usar uma classe:

#### Salvando função de perda customizada definida usando classe Loss

In [55]:
class CosineLoss(keras.losses.Loss):
    
    def __init__(self, classw = np.ones((10,)), **kwargs):
        self.classw = classw
        super().__init__(**kwargs) 
        
    def call(self, y_true, y_pred):
        # fazer a conversao aki eh pessima ideia, mas necessario na versao beta do tf2/keras2
        # que ainda nao eh capaz de salvar parametros que sejam tensores
        v2 = K.constant(self.classw, dtype = tf.float32) * y_pred
        norm_v1 = tf.nn.l2_normalize(y_true, 1)        
        norm_v2 = tf.nn.l2_normalize(v2, 1)
        return 1 - tf.matmul(norm_v1, norm_v2, transpose_b=True) 

    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "classw": self.classw}

In [56]:
model = keras.models.Model(inputs=[X_input], outputs=[yhat])

model.compile(loss = CosineLoss(), optimizer = "adam", metrics = ["accuracy"])

In [57]:
h = model.fit(X_train, y_train_d, epochs = 1, 
             validation_data = (X_valid, y_valid_d))

W0923 13:18:20.133491 4619273664 training_utils.py:1211] When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.


55000/55000 [==============================] - 7s 136us/sample - loss: 0.6828 - accuracy: 0.8182 - val_loss: 0.6831 - val_accuracy: 0.8080


In [58]:
model.save("model_cosine_loss_class.h5")

In [59]:
# para ler
model = keras.models.load_model("model_cosine_loss_class.h5",
                                custom_objects = {"CosineLoss": CosineLoss})

ValueError: Unknown loss function: CosineLoss

## Camadas e Loss: um autocodificador variacional

<img src="imagens/VAE-ammd2-2019.png" alt="drawing" width="1000"/>

In [65]:
class Amostragem(keras.layers.Layer):
    def call(self, inputs):
        z_media, z_log_var = inputs
        batch = tf.shape(z_media)[0]
        dim = tf.shape(z_media)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_media + tf.exp(0.5 * z_log_var) * epsilon

In [67]:
class Codificador(keras.models.Model):
    def __init__(self, latent_dim=32, inter_dim=64,
               name='codificador', **kwargs):
        super(Codificador, self).__init__(name=name, **kwargs)
        self.dense_proj = keras.layers.Dense(inter_dim, activation='relu')
        self.dense_mean = keras.layers.Dense(latent_dim)
        self.dense_log_var = keras.layers.Dense(latent_dim)
        self.amostragem = Amostragem()

    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_media = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.amostragem((z_media, z_log_var))
        return z_media, z_log_var, z

In [68]:
class Decodificador(keras.models.Model):
    def __init__(self, original_dim, inter_dim=64,
               name='decodificador', **kwargs):
        super(Decodificador, self).__init__(name=name, **kwargs)
        self.dense_proj = keras.layers.Dense(inter_dim, activation='relu')
        self.dense_output = keras.layers.Dense(original_dim, activation='sigmoid')

    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)

In [79]:
class AutocodificadorVariacional(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(self, original_dim, inter_dim=64,
               latent_dim=32, name='autoencoder', **kwargs):
        super(AutocodificadorVariacional, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.codificador = Codificador(latent_dim=latent_dim, inter_dim=inter_dim)
        self.decodificador = Decodificador(original_dim, inter_dim=inter_dim)
        self.opt = tf.keras.optimizers.Adam(learning_rate=1e-3)

    def call(self, inputs):
        z_media, z_log_var, z = self.codificador(inputs)
        rec = self.decodificador(z)
        # Add KL divergence regularization loss.
        kl_loss = - 0.5 * tf.reduce_mean(z_log_var - tf.square(z_media) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return rec

In [75]:
X_train_flat = X_train.reshape(-1, 784)
X_valid_flat = X_valid.reshape(-1, 784)

In [76]:
vae = AutocodificadorVariacional(784, 64, 32)

In [77]:
vae.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

In [78]:
h = vae.fit(X_train_flat, X_train_flat, 
           epochs = 8, shuffle=True,
           validation_data = (X_valid_flat, X_valid_flat))

W0925 14:02:09.559137 4619273664 training_utils.py:1211] When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.


Epoch 1/8
55000/55000 [==============================] - 4s 81us/sample - loss: 0.0265 - val_loss: 0.0178
Epoch 2/8
55000/55000 [==============================] - 4s 69us/sample - loss: 0.0163 - val_loss: 0.0151
Epoch 3/8
55000/55000 [==============================] - 4s 70us/sample - loss: 0.0144 - val_loss: 0.0139
Epoch 4/8
55000/55000 [==============================] - 4s 70us/sample - loss: 0.0133 - val_loss: 0.0129
Epoch 5/8
55000/55000 [==============================] - 4s 69us/sample - loss: 0.0127 - val_loss: 0.0123
Epoch 6/8
55000/55000 [==============================] - 4s 70us/sample - loss: 0.0122 - val_loss: 0.0120
Epoch 7/8
55000/55000 [==============================] - 4s 70us/sample - loss: 0.0119 - val_loss: 0.0117
Epoch 8/8
55000/55000 [==============================] - 4s 70us/sample - loss: 0.0116 - val_loss: 0.0114


In [88]:
class AutocodificadorVariacional(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(self, original_dim, inter_dim=64,
               latent_dim=32, name='autoencoder', **kwargs):
        super(AutocodificadorVariacional, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim
        self.codificador = Codificador(latent_dim=latent_dim, inter_dim=inter_dim)
        self.decodificador = Decodificador(original_dim, inter_dim=inter_dim)
        self.opt = tf.keras.optimizers.Adam(learning_rate=1e-3)
        self.mse_loss = keras.losses.MeanSquaredError()

    def call(self, inputs):
        z_media, z_log_var, z = self.codificador(inputs)
        rec = self.decodificador(z)
        # Add KL divergence regularization loss.
        kl_loss = - 0.5 * tf.reduce_mean(z_log_var - tf.square(z_media) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss + self.mse_loss(inputs, rec))
        return rec

In [89]:
vae = AutocodificadorVariacional(784, 64, 32)

In [92]:
vae.compile(optimizer='adam', loss = lambda y_true, y_pred: tf.constant(0, dtype = tf.float32))

In [93]:
h = vae.fit(X_train_flat, X_train_flat, 
           epochs = 8, shuffle=True,
           validation_data = (X_valid_flat, X_valid_flat))

W0923 12:43:15.622452 4634867136 training_utils.py:1211] When passing input data as arrays, do not specify `steps_per_epoch`/`steps` argument. Please use `batch_size` instead.


Epoch 1/8
55000/55000 [==============================] - 6s 112us/sample - loss: 0.0898 - val_loss: 0.0883
Epoch 2/8
55000/55000 [==============================] - 4s 73us/sample - loss: 0.0874 - val_loss: 0.0881
Epoch 3/8
55000/55000 [==============================] - 4s 75us/sample - loss: 0.0873 - val_loss: 0.0879
Epoch 4/8
55000/55000 [==============================] - 4s 71us/sample - loss: 0.0873 - val_loss: 0.0878
Epoch 5/8
55000/55000 [==============================] - 4s 71us/sample - loss: 0.0872 - val_loss: 0.0878
Epoch 6/8
55000/55000 [==============================] - 4s 70us/sample - loss: 0.0872 - val_loss: 0.0879
Epoch 7/8
55000/55000 [==============================] - 4s 77us/sample - loss: 0.0872 - val_loss: 0.0878
Epoch 8/8
55000/55000 [==============================] - 4s 73us/sample - loss: 0.0871 - val_loss: 0.0878


In [94]:
vae.losses

[<tf.Tensor 'autoencoder/add_1:0' shape=() dtype=float32>]